In [ ]:
from keras.applications import MobileNet

img_rows = 224
img_cols = 224 
 
MobileNet = MobileNet(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

for layer in MobileNet.layers:
    layer.trainable = False
    
for (i,layer) in enumerate(MobileNet.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

In [ ]:
def vt(bottom_model, num_classes):
    
    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024, activation = "relu")(top_model)
    top_model = Dense(1024, activation = "relu")(top_model)
    top_model = Dense(512, activation = "relu")(top_model)
    top_model = Dense(num_classes, activation = "softmax")(top_model)
    return top_model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

#set our class numberv to 3 (Young,Middle,Old)
num_classes = 5

FC_Head = vt(MobileNet, num_classes)

model = Model(inputs=MobileNet.input, outputs=FC_Head)

print(model.summary())

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.models import sequential
from keras.optimizers import RMSprop

model.compile(loss = 'categorical_crossentropy',
             optimizer = 'adam',
             metrics = [ 'accuracy'])

train_data_dir = 'train/'
validation_data_dir = 'test/'

#Let's use some data augmentation
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=45,
      width_shift_range=0.3,
      height_shift_range=0.3,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)

#set our batch size (typically on most mid tier systems we will use 16-32)
batch_size = 32
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')

epochs = 3

batch_size = 16

model.fit_genertaor(
    train_generator,
    steps_Per_epoch = 80,
    validation_data = validation_generator)

In [ ]:
model.save("task.net")

In [ ]:
from keras.models import load_model

calssifier = load_model('task.net')

In [ ]:
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join


face_rec_n = {"[0]": "harshitha",
                         "[1]": "nithish",
                        "[2]": "varshi",
                        "[3]": "vyshu",
                        "[4]": "vyshu_mom"}

face_rec_n = {"harshitha": "harshitha_1",
                        "nithish": "nithish_2",
                         "varshi": "varshi_3",
                         "vyshu": "vyshu_4",
                         "vyshu_mom": "vyshu_mom_5"}

def draw_test(name, pred, im):
    face = face_rec[ str(pred) ]
    Black = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im, 80, 0, 100, cv2,BORDER_CONSTANT, value=BLACK)
    cv2.putText(expanded_iamge, face, (20,60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,225), 2)
    cv2.imshow(name, expanded_image)
    
def getRandomImage(path):
    
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path,x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[ random_directory ]
    print("class - " + face_rec_n[str(path_class)])
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0, len(file_names))
    image_name = file_names[random_fiule_index]
    return cv2.imread(file_path+"/"+image_name)

for i in range(0,10):
    input_im = getrandomImage("test/")
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, (224,224), None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)
    
    input_im = cv2.resize(input_im, (224,224), interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 225.
    input_im = input_im.reshape(1,224,224,3)
    
    #Get prediction
    res = np.argmax(classifier.predict(input_im, 1, verbose = 0), axis = 1)
    
    #Show image with predicted class
    draw_test("Prediction", res, input_original)
    cv2.waitKey(0)
    
cv2.destroyAllWindows()